# FEV vs. Alpha Analysis

For each VGG19 layer, this notebook plots **mean FEV** vs. **alpha** (where alpha is the slope of the log–log PCA variance spectrum).

- **Mean FEV** is taken from the regression: VGG19 features (reduced to N PCs) → Neural data (original dimensionality)

- For each layer, we select the **best FEV** (i.e., highest across PC counts)

In [ ]:
from cortexlib.power_law import PowerLawAlphaEstimator
from cortexlib.mouse import CortexlabMouse
from cortexlib.utils import file as futils
from cortexlib.utils.logging import Logger
from cortexlib.utils.random import set_global_seed, GLOBAL_SEED
from cortexlib.utils.plotting import vgg19_colours
from sklearn.decomposition import PCA
from adjustText import adjust_text
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import torch

logger = Logger()
set_global_seed()

In [ ]:
MOUSE_ID = futils.get_mouse_id()
logger.info(f"This notebook is running for mouse {MOUSE_ID}")

In [ ]:
mouse = CortexlabMouse(mouse_id=MOUSE_ID)

logger.progress(f"Computing null distributions for all neurons in mouse {mouse.id}")
null_srv_all_neurons = mouse.compute_null_all_neurons(n_shuffles=100)
logger.success(f"Null distributions computed")

real_srv_all_neurons = mouse.compute_real_srv_all_neurons()
reliable_neuron_indices = mouse.get_reliable_neuron_indices(
            null_srv_all_neurons, real_srv_all_neurons, percentile_threshold=99)
neural_responses_mean, neural_responses, _ = mouse.get_responses_for_reliable_neurons(reliable_neuron_indices, real_srv_all_neurons, num_neurons=500)

logger.info(f"Neural responses shape: {neural_responses.shape}")

In [ ]:
fev_results = futils.read_json_file_as_dataframe('./prediction.json')
best_fev_per_layer = fev_results.loc[fev_results.groupby("layer")["mean_fev"].idxmax()]

best_fev_per_layer

In [ ]:
vgg19_data = futils.load_model_features(futils.Model.VGG19, MOUSE_ID)
vgg19_feats = vgg19_data['features']

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import torch
import random

raw_features = vgg19_feats
layer_names = list(raw_features.keys())
subsample_size = 128
thresholds = [0.68, 0.80, 0.90]
n_repeats = 20  # number of subsamples per layer
random.seed(GLOBAL_SEED)

def run_pca(data, n_components=subsample_size):
    pca = PCA(n_components=n_components, random_state=GLOBAL_SEED)
    pca.fit(data)
    return np.cumsum(pca.explained_variance_ratio_), pca.explained_variance_ratio_

def num_components_for_variance(cumulative_ev, threshold):
    above = np.where(cumulative_ev >= threshold)[0]
    return int(above[0] + 1) if len(above) > 0 else None

def subsample_features(feats, indices):
    if feats.ndim > 2:
        feats = feats.reshape(feats.shape[0], -1)
    return feats[:, indices] if feats.shape[1] >= len(indices) else feats

json_data = []

for layer in layer_names:
    layer_data = {
        "mouse_id": MOUSE_ID,
        "model": futils.get_model(),
        "layer": layer,
        "original_dimensions": int(np.prod(raw_features[layer].shape[1:])),
        "subsampled_dimensions": subsample_size,
        "subsamples": []
    }

    for _ in range(n_repeats):
        rand_indices = torch.randperm(subsample_size)[:subsample_size]
        feats = subsample_features(raw_features[layer], rand_indices)
        cum_ev, _ = run_pca(feats)

        layer_data["subsamples"].append({
            "num_principal_components_68pc_ev": num_components_for_variance(cum_ev, 0.68),
            "num_principal_components_80pc_ev": num_components_for_variance(cum_ev, 0.80),
            "num_principal_components_90pc_ev": num_components_for_variance(cum_ev, 0.90),
            "cumulative_ev": cum_ev.tolist()
        })

    json_data.append(layer_data)

futils.write_json_file(json_data, './explained_variance.json')

In [ ]:
full_dim = []

for layer in layer_names:
    layer_data = {
        "mouse_id": MOUSE_ID,
        "model": futils.get_model(),
        "layer": layer,
        "original_dimensions": int(np.prod(raw_features[layer].shape[1:])),
    }

    feats = raw_features[layer]
    if feats.ndim > 2:
        feats = feats.reshape(feats.shape[0], -1)
    cum_ev, _ = run_pca(feats)
    layer_data["num_principal_components_68pc_ev"] = num_components_for_variance(cum_ev, 0.68)
    layer_data["num_principal_components_80pc_ev"] = num_components_for_variance(cum_ev, 0.80)
    layer_data["num_principal_components_90pc_ev"] = num_components_for_variance(cum_ev, 0.90)
    layer_data["cumulative_ev"] = cum_ev.tolist()
    full_dim.append(layer_data)

futils.write_json_file(full_dim, './explained_variance_full_dim.json')

In [ ]:
power_law_alpha_estimator = PowerLawAlphaEstimator()

def choose_k_window_from_ncomp(ncomp: int):
    """
    Return (kmin, kmax) for power-law fits depending only on
    the number of available principal components (ncomp).
    """

    # drop the head (~5–10% of PCs), keep the middle, avoid noisy tail
    kmin = max(20, int(0.05 * ncomp))     # at least 20 PCs
    kmax = min(int(0.6 * ncomp), ncomp)   # cap at ~60% of spectrum

    return kmin, kmax

alpha_results = []

logger.progress(f"Computing α for VGG19 features across layers")
for layer, feats in tqdm(vgg19_feats.items(), desc="α for all VGG19 layers"):
    logger.progress(f"VGG19 {layer}")

    n_images, C = feats.shape
    ncomp = min(n_images, C)
    kmin, kmax = choose_k_window_from_ncomp(ncomp)

    feats = feats if feats.dim() <= 2 else feats.view(feats.size(0), -1)
    alpha = power_law_alpha_estimator.compute_alpha_rank(
        feats, # (n_images × n_units)
        kmin=kmin, 
        kmax=kmax,
        log_spaced=True, 
        npts=60
    )

    alpha_results.append({
        'layer': layer,
        'alpha': alpha.alpha,
        'alpha_no_pc1': alpha.alpha_no_pc1,
    })

logger.success(f"Alphas computed")

In [ ]:
futils.write_json_file(alpha_results, './dimensionality.json')

In [ ]:
alpha_results_df = pd.DataFrame(alpha_results)
alpha_results_df

In [ ]:
merged_results = pd.merge(best_fev_per_layer, alpha_results_df, on='layer')
merged_results

In [ ]:
colours = merged_results['layer'].map(vgg19_colours)

plt.figure(figsize=(10, 6))
plt.scatter(merged_results['alpha'], merged_results['mean_fev'], c=colours)

texts = []
for _, row in merged_results.iterrows():
    texts.append(
        plt.text(row['alpha'], row['mean_fev'], row['layer'],
                 color=vgg19_colours[row['layer']], fontsize=10)
    )

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray'))

plt.xlabel('Alpha (Power Law Exponent)')
plt.ylabel('Mean FEV')
plt.title('VGG19 Layers: Alpha vs. Predictive Power (Neural Data)')
plt.grid(True)
plt.tight_layout()
plt.show()